Utilizando el dataset " Twitter Sentiment Analysis in Spanish Tweets", deberás
implementar y comparar diferentes técnicas de Inteligencia Artificial y Modelos de
Lenguaje (LLM) para la clasificación de sentimientos en comentarios de usuarios.
• Demostrar conocimientos prácticos en técnicas de IA/ML
• Evaluar capacidad de prompt engineering con LLMs
• Analizar críticamente los resultados obtenidos
• Muestra a utilizar: Los primeros 100 datos seleccionados aleatoriamente
• Etiquetas esperadas: POSITIVO, NEGATIVO, NEUTRO


## Plan de reentrenamiento reproducible
1. Preparar entorno limpio (usa `venv` o `conda`) e instala solo las dependencias necesarias (`pandas`, `numpy`, `scikit-learn`, `pysentimiento`, `joblib`, `tarfile` ya viene con Python). Mantén la misma versión de Python que usarás en `deploy.ipynb`.
2. Descargar y descomprimir el dataset "Twitter Sentiment Analysis in Spanish Tweets". Si vuelves a muestrear, fija la semilla para poder replicar (ej. `random_state=42`).
3. Limpieza y preprocesado: normaliza texto, elimina duplicados si aparecen y mapea las etiquetas a `POSITIVO`, `NEGATIVO`, `NEUTRO` como se usa en las métricas.
4. División de datos: separa en train/test (y valid si lo necesitas) manteniendo estratificación para no perder la proporción de clases.
5. Entrenamiento comparativo: entrena Logistic Regression, MultinomialNB y Linear SVM con `CountVectorizer` y `TfidfVectorizer`. Guarda las métricas (accuracy, f1-macro y por clase) en tablas reproducibles.
6. Interpretación: documenta por qué eliges el modelo final (actualmente Linear SVM + CountVectorizer). Conserva también el `vectorizer` para inferencia.
7. Serialización: guarda el pipeline completo (`joblib.dump`) y arma `model.tar.gz` con el artefacto y el script de inferencia usado por `deploy.ipynb`.
8. Congela las dependencias antes de subir a S3 ejecutando la celda "Registro de dependencias y freeze"; copia `requirements_sentimientos.txt` junto al `model.tar.gz` para SageMaker y evita errores de versión en CloudWatch.


In [ ]:
#%pip install pandas scikit-learn pysentimiento  jupyter ipywidgets

#%pip install --upgrade pip





: 

In [ ]:
import pandas as pd

df = pd.read_csv("sentiment_analysis_dataset.csv")
df.head()

In [ ]:
print("Categorías en 'emotion':", df['emotion'].unique())
print("Categorías en 'sentiment':", df['sentiment'].unique())

In [ ]:
# Diccionarios de mapeo
emotion_map = {
    'overwhelmed': 'NEGATIVO', 'embarrassed': 'NEGATIVO', 'jealous': 'NEGATIVO',
    'irritated': 'NEGATIVO', 'frustrated': 'NEGATIVO', 'distant': 'NEGATIVO',
    'stupid': 'NEGATIVO', 'isolated': 'NEGATIVO', 'sleepy': 'NEGATIVO',

    'responsive': 'NEUTRO', 'relaxed': 'NEUTRO',

    'loving': 'POSITIVO', 'thankful': 'POSITIVO', 'secure': 'POSITIVO',
    'confident': 'POSITIVO', 'successful': 'POSITIVO', 'surprised': 'POSITIVO',
    'playful': 'POSITIVO', 'optimistic': 'POSITIVO', 'daring': 'POSITIVO'
}

sentiment_map = {
    'scared': 'NEGATIVO', 'mad': 'NEGATIVO', 'sad': 'NEGATIVO',
    'peaceful': 'NEUTRO',
    'powerful': 'POSITIVO', 'joyful': 'POSITIVO'
}

# Combinar ambos mapas para una sola columna final
def combine_sentiment(row):
    e = emotion_map.get(row['emotion'], 'NEUTRO')
    s = sentiment_map.get(row['sentiment'], 'NEUTRO')
    # Regla: si alguno es NEGATIVO => NEGATIVO; si alguno es POSITIVO => POSITIVO
    if 'NEGATIVO' in (e, s):
        return 'NEGATIVO'
    elif 'POSITIVO' in (e, s):
        return 'POSITIVO'
    else:
        return 'NEUTRO'

df['sentiment_label'] = df.apply(combine_sentiment, axis=1)

# Verifica el resultado
print(df[['text', 'emotion', 'sentiment', 'sentiment_label']].sample(10))
print(df['sentiment_label'].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

import joblib
import os
# usar dataset completo

df_sample = df.copy()  # usar todo el dataset
print(df_sample['sentiment_label'].value_counts())


# Selecciona el texto y la etiqueta
X = df_sample['text']
y = df_sample['sentiment_label']

# Divide en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Prueba ambas vectorizaciones
vectorizers = {
    "CountVectorizer": CountVectorizer(),
    "TfidfVectorizer": TfidfVectorizer()
}

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "Linear SVM": LinearSVC()
}

# Crear directorios para guardar los modelos
os.makedirs('models/svm_countvectorizer', exist_ok=True)
os.makedirs('models/svm_tfidfvectorizer', exist_ok=True)

for vec_name, vectorizer in vectorizers.items():
    print(f"\n--- Usando {vec_name} ---")
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    for model_name, model in models.items():
        print(f"\nModelo: {model_name}")
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)
        
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')
        cm = confusion_matrix(y_test, y_pred, labels=['POSITIVO', 'NEGATIVO', 'NEUTRO'])
        
        print(f"Accuracy: {acc:.4f}")
        print(f"F1-score (macro): {f1:.4f}")
        print("Matriz de confusión:")
        print(cm)
        print("Reporte de clasificación:")
        print(classification_report(y_test, y_pred, digits=4))

        # Guardar solo los modelos Linear SVM y sus vectorizadores
        if model_name == "Linear SVM":
            if vec_name == "CountVectorizer":
                joblib.dump(model, 'models/svm_countvectorizer/model.joblib')
                joblib.dump(vectorizer, 'models/svm_countvectorizer/vectorizer.joblib')
            elif vec_name == "TfidfVectorizer":
                joblib.dump(model, 'models/svm_tfidfvectorizer/model.joblib')
                joblib.dump(vectorizer, 'models/svm_tfidfvectorizer/vectorizer.joblib')

## Interpretación de resultados y modelo recomendado
Se compararon tres modelos de clasificación de sentimientos (Logistic Regression, Naive Bayes y Linear SVM) usando dos técnicas de vectorización de texto (CountVectorizer y TfidfVectorizer).
- **CountVectorizer**: Linear SVM obtuvo la mejor precisión (Accuracy: 0.82, F1 macro: 0.77), mostrando buen desempeño en las clases POSITIVO y NEGATIVO, aunque la clase NEUTRO fue la más difícil de predecir (menor recall y f1-score).
- **TfidfVectorizer**: Linear SVM también fue el mejor (Accuracy: 0.81, F1 macro: 0.71), pero la clase NEUTRO sigue siendo la menos representada correctamente.
En general, **Linear SVM con CountVectorizer** fue el modelo más robusto, logrando el mejor balance entre precisión y F1-score macro. Sin embargo, todos los modelos presentan dificultades para clasificar correctamente la clase NEUTRO, posiblemente por desbalance de clases o menor información en los textos asociados.
**Conclusión:** El modelo recomendado es **Linear SVM con CountVectorizer**, ya que ofrece el mejor desempeño global en este problema de clasificación de sentimientos en tweets en español.

In [ ]:
print(X_test.shape)
#print(X_test,y_test)
print(X_test.tolist(),y_test.tolist())

In [ ]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["ACCELERATE_USE_CPU"] = "true"
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es", device="cpu")
resultado = analyzer.predict(X_test.iloc[0])
print(resultado)

In [ ]:
# Función para mapear las etiquetas de pysentimiento a nuestro formato
def map_pysentimiento_label(pysentimiento_label):
    """
    Mapea las etiquetas de pysentimiento a nuestro formato
    NEU -> NEUTRO
    POS -> POSITIVO  
    NEG -> NEGATIVO
    """
    mapping = {
        'NEU': 'NEUTRO',
        'POS': 'POSITIVO', 
        'NEG': 'NEGATIVO'
    }
    return mapping.get(pysentimiento_label, 'NEUTRO')

# Función para formatear texto largo con saltos de línea
def format_long_text(text, max_width=80):
    """Divide el texto en líneas de máximo max_width caracteres"""
    import textwrap
    return '\n'.join(textwrap.wrap(text, width=max_width))

# Ejemplo con el resultado actual - Versión mejorada para mejor visualización
print("=" * 60)
print("ANÁLISIS DE MUESTRA DE PRUEBA")
print("=" * 60)

print("\n📝 TEXTO DE PRUEBA:")
print("-" * 40)
print(format_long_text(X_test.iloc[0]))

print(f"\n🎯 ETIQUETA REAL: {y_test.iloc[0]}")

print(f"\n🤖 PREDICCIÓN PYSENTIMIENTO:")
print(f"   • Etiqueta original: {resultado.output}")
print(f"   • Etiqueta mapeada: {map_pysentimiento_label(resultado.output)}")

print(f"\n📊 PROBABILIDADES:")
for label, prob in resultado.probas.items():
    mapped_label = map_pysentimiento_label(label)
    print(f"   • {mapped_label}: {prob:.4f}")

print("\n" + "=" * 60)

In [ ]:
# Procesar todas las muestras de prueba con pysentimiento
print("Procesando", len(X_test), "muestras de prueba...")
print("Esto puede tomar unos momentos...")

y_pred_pysentimiento = []
probabilidades_pysentimiento = []

for i, texto in enumerate(X_test):
    print(f"Procesando muestra {i+1}/{len(X_test)}", end='\r')
    
    # Hacer predicción
    resultado = analyzer.predict(texto)
    
    # Extraer etiqueta predicha y mapearla
    etiqueta_predicha = map_pysentimiento_label(resultado.output)
    y_pred_pysentimiento.append(etiqueta_predicha)
    
    # Guardar probabilidades (opcional, para análisis posterior)
    probabilidades_pysentimiento.append(resultado.probas)

print("\n¡Predicciones completadas!")
print(f"Total de predicciones: {len(y_pred_pysentimiento)}")
print(f"Primeras 5 predicciones: {y_pred_pysentimiento[:5]}")
print(f"Primeras 5 etiquetas reales: {y_test.tolist()[:5]}")

In [ ]:
# Evaluación del modelo pysentimiento
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np

# Convertir y_test a lista para la comparación
y_test_list = y_test.tolist()

print("=== EVALUACIÓN DEL MODELO PYSENTIMIENTO ===")
print(f"Muestras evaluadas: {len(y_test_list)}")
print()

# Métricas principales
accuracy = accuracy_score(y_test_list, y_pred_pysentimiento)
f1_macro = f1_score(y_test_list, y_pred_pysentimiento, average='macro')
f1_weighted = f1_score(y_test_list, y_pred_pysentimiento, average='weighted')

print(f"Accuracy (Precisión): {accuracy:.4f}")
print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")
print()

# Matriz de confusión
labels = ['POSITIVO', 'NEGATIVO', 'NEUTRO']
cm = confusion_matrix(y_test_list, y_pred_pysentimiento, labels=labels)
print("Matriz de Confusión:")
print("Filas: Etiquetas reales, Columnas: Predicciones")
print("                 POSITIVO  NEGATIVO  NEUTRO")
for i, label in enumerate(labels):
    print(f"{label:>12} {cm[i][0]:>8} {cm[i][1]:>8} {cm[i][2]:>8}")
print()

# Reporte detallado de clasificación
print("Reporte de Clasificación Detallado:")
print(classification_report(y_test_list, y_pred_pysentimiento, labels=labels, digits=4))

In [ ]:
# Comparación con modelos anteriores
print("=== COMPARACIÓN DE MODELOS ===")
print()

# Resultados de modelos anteriores (de tu análisis previo)
resultados_modelos = {
    "Linear SVM + CountVectorizer": {"accuracy": 0.82, "f1_macro": 0.77},
    "Linear SVM + TfidfVectorizer": {"accuracy": 0.81, "f1_macro": 0.71},
    "PysentimientoLLM": {"accuracy": accuracy, "f1_macro": f1_macro}
}

print("Modelo                      Accuracy  F1-macro")
print("-" * 50)
for modelo, metricas in resultados_modelos.items():
    print(f"{modelo:<25} {metricas['accuracy']:>8.4f} {metricas['f1_macro']:>8.4f}")

print()
print("=== CONCLUSIONES ===")
print("1. PysentimientoLLM obtuvo una accuracy de {:.1f}% y F1-macro de {:.1f}%".format(accuracy*100, f1_macro*100))
print("2. Los modelos tradicionales (SVM) superaron significativamente al LLM en esta muestra")
print("3. El modelo PysentimientoLLM tuvo mejor desempeño en NEGATIVO que en POSITIVO y NEUTRO")
print("4. La clase NEUTRO sigue siendo la más difícil de clasificar para todos los modelos")

# Análisis de probabilidades (opcional)
print()
print("=== ANÁLISIS DE CONFIANZA (PROBABILIDADES) ===")
confianzas = []
for probs in probabilidades_pysentimiento:
    max_prob = max(probs.values())
    confianzas.append(max_prob)

print(f"Confianza promedio: {np.mean(confianzas):.3f}")
print(f"Confianza mínima: {np.min(confianzas):.3f}")
print(f"Confianza máxima: {np.max(confianzas):.3f}")
print(f"Predicciones con alta confianza (>0.7): {sum(1 for c in confianzas if c > 0.7)}/{len(confianzas)}")


# Empaquetado de Modelos para SageMaker
- Verifica que el pipeline elegido (modelo + vectorizador) quede en un archivo único (`model.pkl`/`joblib`), listo para cargar en el handler de `deploy.ipynb`.
- Arma `model.tar.gz` con el artefacto, cualquier archivo auxiliar (tokenizers, diccionarios) y el script de inferencia que SageMaker espera.
- Ejecuta la celda de registro de dependencias para generar `requirements_sentimientos.txt`; úsalo en la imagen/notebook de despliegue para instalar versiones idénticas a las de entrenamiento.
- Sube ambos (`model.tar.gz` y `requirements_sentimientos.txt`) al bucket S3 y referencia el requirements en tu contenedor o paso de instalación para evitar incompatibilidades en CloudWatch.



## Registro de dependencias y freeze (ejecutar después de entrenar)
Esta celda captura versiones clave y genera `requirements_sentimientos.txt` con `pip freeze`. Úsalo tanto para volver a entrenar como para el contenedor de inferencia en SageMaker.


In [ ]:

import importlib
import json
import platform
import subprocess
import sys

def pkg_version(name):
    try:
        return importlib.import_module(name).__version__
    except Exception:
        return "no encontrado"

summary = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "packages": {
        "pandas": pkg_version("pandas"),
        "numpy": pkg_version("numpy"),
        "scikit-learn": pkg_version("sklearn"),
        "pysentimiento": pkg_version("pysentimiento"),
        "joblib": pkg_version("joblib"),
    },
}

freeze_output = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
with open("requirements_sentimientos.txt", "w", encoding="utf-8") as f:
    f.write(freeze_output)

print(json.dumps(summary, indent=2, ensure_ascii=False))
print("Archivo requirements_sentimientos.txt generado en el directorio actual")


In [ ]:
import tarfile
import os
import shutil
import subprocess
import sys
from pathlib import Path

def prepare_code(src_dir: Path, dest_dir: Path):
    dest_dir.mkdir(parents=True, exist_ok=True)
    for fname in ["inference.py", "__init__.py", "requirements.txt"]:
        shutil.copy(src_dir / fname, dest_dir / fname)

def freeze_requirements(output_path: str = "models/requirements_sentimientos.txt"):
    """Genera un pip freeze de la sesión actual para reproducibilidad."""
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"Generando freeze de dependencias en {output_path}...")
    freeze = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
    output_path.write_text(freeze)
    print("Freeze generado.")

# Congelar dependencias del entorno actual
freeze_requirements()

# --- 1. Empaquetar Linear SVM + CountVectorizer ---
count_dir = Path('models/svm_countvectorizer')
prepare_code(Path('modelos/sentimientos/svm_countvectorizer/code'), count_dir / 'code')
output_filename = 'models/model_svm_countvectorizer.tar.gz'

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(count_dir / 'model.joblib', arcname='model.joblib')
    tar.add(count_dir / 'vectorizer.joblib', arcname='vectorizer.joblib')
    for fname in ["inference.py", "__init__.py", "requirements.txt"]:
        tar.add(count_dir / 'code' / fname, arcname=f'code/{fname}')

print(f"Modelo {output_filename} creado exitosamente.")

# --- 2. Empaquetar Linear SVM + TfidfVectorizer ---
tfidf_dir = Path('models/svm_tfidfvectorizer')
prepare_code(Path('modelos/sentimientos/svm_tfidfvectorizer/code'), tfidf_dir / 'code')
output_filename = 'models/model_svm_tfidfvectorizer.tar.gz'

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(tfidf_dir / 'model.joblib', arcname='model.joblib')
    tar.add(tfidf_dir / 'vectorizer.joblib', arcname='vectorizer.joblib')
    for fname in ["inference.py", "__init__.py", "requirements.txt"]:
        tar.add(tfidf_dir / 'code' / fname, arcname=f'code/{fname}')

print(f"Modelo {output_filename} creado exitosamente.")

# --- 3. Empaquetar PysentimientoLLM ---
# Pysentimiento descarga los modelos en un caché. Intentamos usar una copia local
# y, si no existe, descargamos y empaquetamos desde el caché de HuggingFace.
from pysentimiento import create_analyzer

print("Preparando artefacto de pysentimiento...")
local_model_dir = Path('modelos/sentimientos/model_pysentimiento')
hf_cache_dir = Path.home() / '.cache' / 'huggingface' / 'hub'
model_name = 'pysentimiento/robertuito-sentiment-analysis'

# Preferir copia local (incluida en el repo)
model_source_path = None
if (local_model_dir / 'config.json').exists() and any(local_model_dir.glob('*.safetensors')):
    model_source_path = local_model_dir
else:
    for path in hf_cache_dir.glob(f"models--{model_name.replace('/', '--')}/snapshots/*"):
        if path.is_dir():
            model_source_path = path
            break
    if model_source_path is None:
        # Fuerza la descarga si no estaba en caché
        analyzer = create_analyzer(task="sentiment", lang="es")
        for path in hf_cache_dir.glob(f"models--{model_name.replace('/', '--')}/snapshots/*"):
            if path.is_dir():
                model_source_path = path
                break

# Preparar código de inferencia para pysentimiento
pysent_dir = Path('models/model_pysentimiento')
prepare_code(Path('modelos/sentimientos/model_pysentimiento/code'), pysent_dir / 'code')

output_filename = 'models/model_pysentimiento.tar.gz'
if model_source_path:
    with tarfile.open(output_filename, "w:gz") as tar:
        print(f"Empaquetando modelo desde {model_source_path}")
        tar.add(model_source_path, arcname='model')
        for fname in ["inference.py", "__init__.py", "requirements.txt"]:
            tar.add(pysent_dir / 'code' / fname, arcname=f'code/{fname}')
    print(f"Modelo {output_filename} creado exitosamente.")
else:
    raise FileNotFoundError("No se encontró el modelo de pysentimiento ni en local ni en caché. Ejecuta create_analyzer antes de empaquetar.")


#Comparación Completa de Modelos y Análisis Final
## Comparación de Rendimiento de Modelos
Modelo	Accuracy	F1-macro
Linear SVM + CountVectorizer	0.8200	0.7700
Linear SVM + TfidfVectorizer	0.8100	0.7100
PysentimientoLLM	0.4500	0.4045
Análisis de Confianza (PysentimientoLLM)
Confianza promedio: 0.745
Confianza mínima: 0.454
Confianza máxima: 0.981
Predicciones con alta confianza (>0.7): 12/20
# Conclusiones del Análisis
###1. Rendimiento General de los Modelos
PysentimientoLLM obtuvo una accuracy de 45.0% y F1-macro de 40.4%, lo que representa un rendimiento considerablemente inferior comparado con los modelos tradicionales. Los modelos tradicionales (SVM) superaron significativamente al LLM en esta muestra, con Linear SVM + CountVectorizer alcanzando 82% de accuracy versus 45% del modelo de lenguaje, una diferencia de 37 puntos porcentuales.

### 2. Análisis por Clase de Sentimiento
El modelo PysentimientoLLM tuvo mejor desempeño en NEGATIVO que en POSITIVO y NEUTRO, con los siguientes resultados por clase:

NEGATIVO: F1-score = 0.6316 (mejor desempeño)
NEUTRO: F1-score = 0.4000 (desempeño intermedio)
POSITIVO: F1-score = 0.1818 (mayor dificultad)
Contrario a lo observado en los modelos SVM donde NEUTRO era la clase más problemática, para PysentimientoLLM la clase POSITIVO es la más difícil de clasificar, no NEUTRO. Esto sugiere que el modelo pre-entrenado tiene sesgos específicos hacia la detección de sentimientos negativos.

### 3. Problema de Overconfidence
Un hallazgo crítico es que el modelo PysentimientoLLM muestra alta confianza promedio (74.5%) pero baja precisión (45%). Esto indica un problema de "overconfidence": el modelo está muy seguro de predicciones que resultan ser incorrectas. De las 20 muestras evaluadas, 12 tuvieron alta confianza (>0.7), pero solo 9 fueron clasificadas correctamente.

### 4. Ranking de Dificultad por Clase
Para el modelo PysentimientoLLM, el ranking de dificultad es:

POSITIVO: F1-score = 0.1818 (MÁS DIFÍCIL)
NEUTRO: F1-score = 0.4000 (INTERMEDIO)
NEGATIVO: F1-score = 0.6316 (MÁS FÁCIL)
Recomendaciones y Consideraciones
Modelo Recomendado
Linear SVM con CountVectorizer es el modelo recomendado para este dataset específico, ofreciendo el mejor balance entre accuracy (82%) y F1-score macro (77%).

#### Consideraciones sobre LLMs
Aunque PysentimientoLLM es un modelo pre-entrenado sofisticado, su rendimiento inferior en este caso específico sugiere que:

**Los modelos tradicionales pueden ser más efectivos para datasets pequeños** y bien estructurados
El fine-tuning específico del dominio podría ser necesario para mejorar el rendimiento del LLM
La eficiencia computacional de los modelos SVM es superior para esta tarea
El prompt engineering más sofisticado podría mejorar los resultados del modelo de lenguaje
Implicaciones Prácticas
Para implementaciones en producción con este tipo de datos en español, los modelos tradicionales de machine learning siguen siendo una opción robusta y eficiente, especialmente cuando se cuenta con datasets etiquetados de calidad y recursos computacionales limitados.